In [1]:
#  Gerekli Kütüphaneler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
import pickle
import os

#  1. Veriyi Yükle
df = pd.read_excel("Fit_Data.xlsx")

# 2. Gereksiz Sütunları Kaldır
if "id" in df.columns:
    df.drop(columns=["id"], inplace=True)

#  3. Kategorik Verileri Sayısallaştır
mappings = {
    "Physical Activity Level": {"Sedentary": 0, "Lightly Active": 1, "Moderately Active": 2, "Very Active": 3, "Super Active": 4},
    "Daily Activity Level": {"İyi": 2, "Orta": 1, "Kötü": 0, "Poor": 0, "Moderate": 1, "Good": 2},
    "Prior Exercise Experience": {"Bir yıldan az": 0, "Bir yıl": 1, "Bir yıldan fazla": 2, "Less than a year": 0, "One year": 1, "More than a year": 2},
    "Sleep Quality": {"Excellent": 3, "Good": 2, "Fair": 1, "Poor": 0},
    "Goal": {"Kilo vermek": 0, "Kilo almak": 1, "Kas yapmak": 2, "Lose weight": 0, "Gain weight": 1, "Build muscle": 2}
}

gender_encoder = LabelEncoder()
df["Gender"] = gender_encoder.fit_transform(df["Gender"].astype(str))

for column, mapping in mappings.items():
    if column in df.columns:
        df[column] = df[column].map(mapping)

#  4. Hesaplanan Sütunlar
# BMI
df["BMI"] = (df["Current Weight (lbs)"] * 0.453592) / (df["Height (m)"] ** 2)
# BMR
df["BMR (Calories)"] = np.where(
    df["Gender"] == 1,
    88.362 + (13.397 * df["Current Weight (lbs)"] * 0.453592) + (4.799 * df["Height (m)"] * 100) - (5.677 * df["Age"]),
    447.593 + (9.247 * df["Current Weight (lbs)"] * 0.453592) + (3.098 * df["Height (m)"] * 100) - (4.330 * df["Age"])
)

# Kalori dengesi ve son kilo
calories_per_pound = 3500
df["Daily Caloric Surplus/Deficit"] = (df["Weight Change (lbs)"] * calories_per_pound) / (df["Duration (weeks)"] * 7)
df["Final Weight (lbs)"] = df["Current Weight (lbs)"] + df["Weight Change (lbs)"]

#  5. Eksik Veri Temizliği
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(df.mean(numeric_only=True), inplace=True)
df.fillna(df.mode().iloc[0], inplace=True)

#  6. X, y Ayırımı
target_columns = ["Daily Water Intake (L)", "Daily Exercise (min)", "Daily Calories Consumed"]
X = df.drop(columns=target_columns)
with open("model_columns.pkl", "wb") as f:
    pickle.dump(X.columns.tolist(), f)

y = df[target_columns]

#  7. Eğitim/Test Ayırımı
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#  8. Ölçekleme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

#  9. Model Eğitimi
X_train_copy = np.copy(X_train_scaled)
y_train_copy = y_train.copy().values

models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=50, random_state=42),
    "XGBoost": xgb.XGBRegressor(objective="reg:squarederror", n_estimators=50, random_state=42)
}

results = []
for name, model in models.items():
    model.fit(X_train_copy, y_train_copy)
    preds = model.predict(X_test_scaled)
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    results.append({"Model": name, "MAE": mae, "RMSE": rmse})

best_model_name = sorted(results, key=lambda x: x["RMSE"])[0]["Model"]
final_model = models[best_model_name]

with open("best_model.pkl", "wb") as f:
    pickle.dump(final_model, f)

# 10. Örnek Tahmin
example_user = {
    "Age": 25,
    "Gender": gender_encoder.transform(["Male"])[0],
    "Current Weight (lbs)": 90.0,
    "Weight Change (lbs)": -10.0,
    "Duration (weeks)": 8,
    "Physical Activity Level": mappings["Physical Activity Level"]["Very Active"],
    "Sleep Quality": mappings["Sleep Quality"]["Good"],
    "Stress Level": 3,
    "Goal": mappings["Goal"]["Kilo vermek"],
    "Prior Exercise Experience": mappings["Prior Exercise Experience"]["Bir yıl"],
    "Daily Activity Level": mappings["Daily Activity Level"]["Moderate"],
    "Height (m)": 1.80
}

example_user["BMI"] = (example_user["Current Weight (lbs)"] * 0.453592) / (example_user["Height (m)"] ** 2)
example_user["BMR (Calories)"] = 88.362 + (13.397 * example_user["Current Weight (lbs)"] * 0.453592) + (4.799 * example_user["Height (m)"] * 100) - (5.677 * example_user["Age"])
example_user["Daily Caloric Surplus/Deficit"] = (example_user["Weight Change (lbs)"] * 3500) / (example_user["Duration (weeks)"] * 7)
example_user["Final Weight (lbs)"] = example_user["Current Weight (lbs)"] + example_user["Weight Change (lbs)"]

example_df = pd.DataFrame([example_user])
example_df = example_df[X.columns]
example_scaled = scaler.transform(example_df)
prediction = final_model.predict(example_scaled)

print("\n📢 Tahmin Sonuçları:")
print({
    "Daily Water Intake (L)": round(prediction[0][0], 2),
    "Daily Exercise (min)": round(prediction[0][1], 2),
    "Daily Calories Consumed": round(prediction[0][2], 2)
})



📢 Tahmin Sonuçları:
{'Daily Water Intake (L)': 2.24, 'Daily Exercise (min)': 37.38, 'Daily Calories Consumed': 1864.48}
